# Hyperparameter Tuning for Asian Optioin Deep Hedging

## Intro
시장에 참여하는 투자자들은 옵션의 가격을 효율적으로 계산하는 것과, 포트폴리오의 위험성을 최소한으로 하는 방법에 대한 궁금증에 필연적으로 직면하게 된다. 옵션의 가격을 효율적으로 계 산해야 투자의 성공률을 높일 수 있고, 포트폴리오의 위험성을 효과적으로 제거해야 악영향에 의 해 손해를 보는 상황에서 손해를 최소화할 수 있기 때문이다.

포트폴리오의 위험성을 제거하는 전략으로 폭넓게 쓰이는 것이 Hedging이다. Hedging은 여러 방법으로 포트폴리오에서 위험성을 제거하는 전략이다. Option을 통해 Hedging하는 방식을. Option을 이용한 Hedging은 통계적, 수학적인 공식을 활용하는 전통적인 방식인 Greek Hedging, Delta Hedging이 존재하였으나, 컴퓨터 공학의 발전으로 새로운 Hedging 전략이 등 장하게 되었다. 3세대 Hedging으로도 불리는 Deep Hedging은 AI가 데이터를 기반으로 파생 상 품의 위험성과 가격 책정을 하는 Hedging이다.

3세대 Hedging과 기존 Hedging의 가장 큰 차이점은 3세대 Hedging은 입력 변수들의 관계를 통 해 Option값을 책정하는 중간 계산 과정에만 사용되는 것이 아니라, 입력 변수들의 관계를 가중 치를 통해 조정하고, 이를 통해 가장 최적의 Hedging 결과를 내는 방법을 알아낸다. 즉 어떤 Option Pricing 공식을 주입하여 이용하는 것이 아니라, Option Pricing을 하는 최적의 공식 자체를 만들어내고, 이를 활용하는 것이 3세대 Hedging이라는 것이다.

이 글에서는 2세대, 3세대 Hedging인 Delta Hedging과 Deep Hedging을 KOSPI의 시가총액이 가장 큰 기업 9개를 대상으로 실제로 진행해본다. 진행 결과를 통해 Insight를 얻어보고, Deep Hedging의 성능을 높일 방법에 대해 논의해보겠다.

In [1]:
# import packages
import requests
from scipy.stats import norm
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf

In [2]:
# import DATA
# https://www.data.go.kr/iim/api/selectAPIAcountView.do#layer-api-guide
key = 'pM63raeWJv8s%2BtrbKRC9kS5gkPnGsekcRcCoTCdsT36azCw4PJDGb7QyJlhUWA64Bu%2BCetvU5jUCHFo7rjfQig%3D%3D'
url = 'https://api.odcloud.kr/api/GetStockSecuritiesInfoService/v1/getStockPriceInfo?numOfRows=10000&pageNo=1&resultType=json&beginBasDt=20220111&i&beginMrktTotAmt=30000000000000&serviceKey={api_key}'.format(api_key = key)

response = requests.get(url)
body = response.json()
body = body['response']['body']['items']['item']

In [3]:
# Create DataFrame
df = pd.json_normalize(body)

In [4]:
df = df[['basDt', 'itmsNm','clpr']]
df['basDt'] = df['basDt'].astype('datetime64')
df['clpr'] = df['clpr'].astype('float64')
df = df.sort_values(by=['itmsNm', 'basDt'])

df = df[(df['itmsNm']!='LG에너지솔루션')&(df['itmsNm']!='기아')]

In [10]:
px.line(df, x='basDt', y='clpr', color='itmsNm')

In [13]:
# pivot
StockInfo = pd.pivot_table(df, values='clpr', index='basDt', columns='itmsNm')

In [20]:
display(StockInfo)

itmsNm,LG화학,NAVER,SK하이닉스,삼성SDI,삼성바이오로직스,삼성전자,삼성전자우,카카오,현대차
basDt,,,,,,,,,
2022-01-11,734000.0,335000.0,128000.0,627000.0,833000.0,78900.0,71900.0,95000.0,210500.0
2022-01-12,773000.0,345500.0,128500.0,659000.0,857000.0,78900.0,72400.0,97200.0,211500.0
2022-01-13,755000.0,348000.0,129500.0,666000.0,865000.0,77900.0,72600.0,96700.0,210000.0
2022-01-14,716000.0,342000.0,128500.0,647000.0,850000.0,77300.0,72500.0,93900.0,209000.0
2022-01-17,707000.0,343500.0,127000.0,641000.0,826000.0,77500.0,71900.0,92900.0,205000.0
...,...,...,...,...,...,...,...,...,...
2022-08-26,616000.0,242000.0,95100.0,586000.0,847000.0,60000.0,55100.0,76000.0,193500.0
2022-08-29,604000.0,234000.0,92500.0,576000.0,829000.0,58600.0,54500.0,72200.0,188500.0
2022-08-30,611000.0,238000.0,93200.0,578000.0,836000.0,58800.0,54700.0,73200.0,195500.0


In [ ]:
# definite Call Option, Put Option, Delta, Geometical Average
def bscall(S, K, T, r, sig):
    d1 = (np.log(S/K)+(r+0.5*sig**2)*T)/(sig*np.sqrt(T))
    d2 = (np.log(S/K)+(r-0.5*sig**2)*T)/(sig*np.sqrt(T))
    return S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    
def bsput(S, K, T, r, sig):
    d1 = (np.log(S/K)+(r+0.5*sig**2)*T)/(sig*np.sqrt(T))
    d2 = (np.log(S/K)+(r-0.5*sig**2)*T)/(sig*np.sqrt(T))
    return K*np.exp(-r*T)*norm.cdf(-d2)-S*norm.cdf(-d1)

def GeometricAsian(S, K, r, v, q, T, N):
    dt = T/N
    nu = r - q -0.5*v**2
    a = N * (N+1) * (2.0 * N + 1.0) / 6.0
    V = np.exp(-r*T) * S * np.exp(((N+1)*nu/2 + a*v**2/(2*N**2))*dt)
    vavg = v * np.sqrt(a)/ (np.power(N, 1.5))
    val = bsCall(V, K, r, vavg, q, T)
    return val

def GeometricAsianDelta(S, K, r, v, q, T, N):
    dt = T/N
    nu = r - q -0.5*v**2
    a = N * (N+1) * (2.0 * N + 1.0) / 6.0
    V = np.exp(-r*T) * S * np.exp(((N+1)*nu/2 + a*v**2/(2*N**2))*dt)
    vavg = v * np.sqrt(a)/ (np.power(N, 1.5))

    d1 = (np.log(V/K) + (r*q + 0.5*vavg**2)*T)/(vavg*np.sqrt(T))
    N1 = norm.cdf(d1)

    delta = np.exp(-r*T)*np.exp(-q*T)*N1*np.exp(((N + 1)*nu/2+a*v**2/(2*N**2))*dt)
    return delta

# Fixed Strike Asian Option 에서 Option의 가격을 설정한다.
def call_price(df, column, K):
    avg = np.average(df[column])
    call = avg - K
    if call > 0:
        return call
    else:
        return 0

def put_price(df, column, K):
    avg = np.average(df[column])
    put = K - avg
    if put > 0:
        return put
    else:
        return 0